In [ ]:
#The model architecture

with strategy.scope():


    image_input = Input(shape=(112, 112, 64, 1), name='image_input') 
    q = layers.Rescaling(1.0 / 255)(image_input)
    
    non_image_input = Input(shape=(7,), name='non_image_input')  
    mu_1 = layers.Dense (64 , activation = "relu")(non_image_input)
    mu_2 = layers.Dense (128 , activation = "relu")(mu_1)
    mu_3 = layers.Dense (256 , activation = "relu" )(mu_2)
    mu_4 = layers.Dropout(0.1)(mu_3)


    
    

    a = layers.Conv3D(32, 3, padding='same', kernel_initializer='he_normal', activation='relu')(q)
    a_2 = layers.Conv3D(64, 3, padding='same', kernel_initializer='he_normal')(a)
    a_3 = layers.BatchNormalization()(a_2)
    a_4 = layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding="same")(a_3)

    b = layers.Conv3D(32, 5, padding='same', kernel_initializer='he_normal', activation='relu')(q)
    b_2 = layers.Conv3D(64, 5, padding='same', kernel_initializer='he_normal')(b)
    b_3 = layers.BatchNormalization()(b_2)
    b_4 = layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding="same")(b_3)

    x = layers.concatenate([a_4,b_4]) 

    x_1 = layers.Conv3D(128, 3, padding='same', kernel_initializer='he_normal', activation='relu')(x)
    x_1a = layers.Conv3D(256, 3, padding='same', kernel_initializer='he_normal', activation='relu')(x_1)
    x_1b = layers.BatchNormalization()(x_1a)
    x_1c = layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding="same")(x_1b)

    f_1 = layers.Conv3D(512, 1, padding='same', kernel_initializer='he_normal', activation='relu')(x_1c)
    f_1b = layers.BatchNormalization()(f_1)
    f_1c = layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding="same")(f_1b)

    f_2 = layers.Conv3D(512, 3, padding='same', kernel_initializer='he_normal', activation='relu')(x_1c)
    f_2b = layers.BatchNormalization()(f_2)
    f_2c = layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding="same")(f_2b)

    f_3 = layers.Conv3D(512, 5, padding='same', kernel_initializer='he_normal', activation='relu')(x_1c)
    f_3b = layers.BatchNormalization()(f_3)
    f_3c = layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding="same")(f_3b)


    g = layers.concatenate([f_1c, f_2c,f_3c])
    
    g_1 = layers.Conv3DTranspose(256, 3, strides=(2, 2, 2), padding="same", activation='relu')(g)
    g_1x = layers.Conv3DTranspose(256, 5, strides=(2, 2, 2), padding="same", activation='relu')(g)
    g_1b = layers.concatenate([g_1, g_1x])

    
    g_3 = layers.Conv3D(256, 3, padding='same', kernel_initializer='he_normal', activation='relu')(g_1b)



    g_4 = layers.Conv3DTranspose(128, 3, strides=(2, 2, 2), padding="same", activation='relu')(g_3)
    g_4x = layers.Conv3DTranspose(128, 5, strides=(2, 2, 2), padding="same", activation='relu')(g_3)

    gamma = layers.concatenate([g_4x, g_4])


    g_6 = layers.Conv3D(128, 3, padding='same', kernel_initializer='he_normal', activation='relu')(gamma)



    g_7 = layers.Conv3DTranspose(64, 3, strides=(2, 2, 2), padding="same", activation='relu')(g_6)
    g_7x = layers.Conv3DTranspose(64, 5, strides=(2, 2, 2), padding="same", activation='relu')(g_6)

    xxx = layers.concatenate([g_7, g_7x , a ,b])

    g_8 = layers.Conv3D(64, 3, padding='same', kernel_initializer='he_normal', activation='relu')(xxx)
    g_9 = layers.Conv3D(32, 3, padding='same', kernel_initializer='he_normal', activation='relu')(g_8)

    
    
    other_g = keras.layers.GlobalMaxPooling3D()(g)
    other_gb = keras.layers.GlobalAveragePooling3D()(g)
    sui = layers.concatenate([other_g ,other_gb , mu_4])
    sui_2 = layers.Dense (512 , activation = "relu")(sui)
    sui_3 = layers.Dropout(0.1)(sui_2)
    sui_4 = layers.Dense (256 , activation = "relu")(sui_3)
    sui_5 = layers.Dense (128 , activation = "relu")(sui_4)
    sui_6 = layers.Dense (64 , activation = "relu")(sui_5)
    
    
    

    mask_array_output = layers.Conv3D(1, (1,1,1) , padding="same", activation="sigmoid" , name = 'mask_array_output')(g_9)
    primary_diag_output = Dense(6, activation="softmax", name="primary_diag_output")(sui_6)
    second_prog_output = Dense(1, activation="sigmoid", name="second_prog_output")(sui_6)
    idh1_mut_output = Dense(3, activation="softmax", name="idh1_mut_output")(sui_6)
    idh2_mut_output  = Dense(2, activation="softmax", name="idh2_mut_output")(sui_6)
    mgmt_meth_output  = Dense(4, activation="softmax", name="mgmt_meth_output")(sui_6)




    model = keras.Model(
    inputs=[image_input, non_image_input],
    outputs=[
        mask_array_output,
        primary_diag_output,
        second_prog_output,
        idh1_mut_output,
        idh2_mut_output,
        mgmt_meth_output
    ])


   
   

In [13]:
#Load the trained model

from tensorflow.keras.models import load_model

model = load_model('/kaggle/input/3dvisualnet/multi_output_model.keras', custom_objects={'custom_loss': custom_loss,
                                                                                         'weighted_sparse_categorical_crossentropy':weighted_sparse_categorical_crossentropy,
                                                                                         'weighted_sparse_categorical_crossentropyb':weighted_sparse_categorical_crossentropyb,'iou':iou})

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ image_input (InputLayer)  │ (None, 112, 112, 64,   │              0 │ -                      │
│                           │ 1)                     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling (Rescaling)     │ (None, 112, 112, 64,   │              0 │ image_input[0][0]      │
│                           │ 1)                     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d (Conv3D)           │ (None, 112, 112, 64,   │            896 │ rescaling[0][0]        │
│                           │ 32)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_2 (Conv3D)         │ (None, 112, 112, 64,   │          4,032 │ rescaling[0][0]        │
│                           │ 32)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_1 (Conv3D)         │ (None, 112, 112, 64,   │         55,360 │ conv3d[0][0]           │
│                           │ 64)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_3 (Conv3D)         │ (None, 112, 112, 64,   │        256,064 │ conv3d_2[0][0]         │
│                           │ 64)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 112, 112, 64,   │            256 │ conv3d_1[0][0]         │
│ (BatchNormalization)      │ 64)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 112, 112, 64,   │            256 │ conv3d_3[0][0]         │
│ (BatchNormalization)      │ 64)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d             │ (None, 56, 56, 32, 64) │              0 │ batch_normalization[0… │
│ (MaxPooling3D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d_1           │ (None, 56, 56, 32, 64) │              0 │ batch_normalization_1… │
│ (MaxPooling3D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 56, 56, 32,     │              0 │ max_pooling3d[0][0],   │
│                           │ 128)                   │                │ max_pooling3d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_4 (Conv3D)         │ (None, 56, 56, 32,     │        442,496 │ concatenate[0][0]      │
│                           │ 128)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_5 (Conv3D)         │ (None, 56, 56, 32,     │        884,992 │ conv3d_4[0][0]         │
│                           │ 256)                   │                │                        │
├──────────────────────

 Total params: 283,293,749 (1.06 GB)

 Trainable params: 94,429,969 (360.22 MB)

 Non-trainable params: 3,840 (15.00 KB)

 Optimizer params: 188,859,940 (720.44 MB)